In [1]:
import apache_beam as beam
p = beam.Pipeline()
pass
p.run()

In [3]:
import apache_beam as beam


with beam.Pipeline() as pipeline:
    pass

In [34]:
import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
#from apache_beam.utils.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)

In [15]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

with beam.Pipeline(InteractiveRunner(),options=options) as p:
    stage1 = p|"Input" >> beam.Create([1,2,3,4,5,6])
    stage2 = stage1| "Multiply">>beam.Map(lambda x: 10*x)
    
ib.show(stage2)

In [42]:
#Usage of Maps

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

def toFloat(x):
    test_list = list(map(float, x))
    return test_list

def makeRow(x):
    row = dict(zip(('v1','v2','v3','v4','v5','v6'),x))
    return row

with beam.Pipeline(InteractiveRunner(),options=options) as p:
    stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv')
    stage2 = stage1| "Split">>beam.Map(lambda x: x.split(','))
    stage3 = stage2| "Conversion to Float">>beam.Map(toFloat)
    stage4 = stage3| "Make a bigquery row">>beam.Map(makeRow)
    
ib.show_graph(p)    
ib.show(stage1)
ib.show(stage2)
ib.show(stage3)
ib.show(stage4)


/usr/bin/dot


In [57]:
#Usage of FlatMap, notice the output in stage3, it is unpacking a list. one to many output

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

def toFloat(x):
    test_list = list(map(float, x))
    return test_list

def makeRow(x):
    row = dict(zip(('v1','v2','v3','v4','v5','v6'),x))
    return row

with beam.Pipeline(InteractiveRunner(),options=options) as p:
    stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv')
    stage2 = stage1| "Split">>beam.Map(lambda x: x.split(','))
    stage3 = stage2| "Conversion to Float">>beam.FlatMap(toFloat)
    #stage4 = stage3| "Make a bigquery row">>beam.Map(makeRow)
    
ib.show_graph(p)    
ib.show(stage1)
ib.show(stage2)
ib.show(stage3)
#ib.show(stage4)


/usr/bin/dot


In [50]:
#This even works without Dataflow Runner

import apache_beam as beam
from apache_beam.runners import DataflowRunner

def toFloat(x):
    test_list = list(map(float, x))
    return test_list

def makeRow(x):
    row = dict(zip(('v1','v2','v3','v4','v5','v6'),x))
    return row

#p = beam.Pipeline()

with beam.Pipeline(options=options) as p:
    stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv')
    stage2 = stage1| "Split">>beam.Map(lambda x: x.split(','))
    stage3 = stage2| "Conversion to Float">>beam.Map(toFloat)
    stage4 = stage3| "Make a bigquery row">>beam.Map(makeRow)
    stage5 = stage4| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='v1:FLOAT,v2:FLOAT,v3:FLOAT,v4:FLOAT,v5:FLOAT,v6:FLOAT',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
#ib.show_graph(p)    
#ib.show(stage1)
#ib.show(stage2)
#ib.show(stage3)
#ib.show(stage4)
#pipeline_result = DataflowRunner().run_pipeline(p, options=options)

In [54]:
# Dataflow Runner

import apache_beam as beam
from apache_beam.runners import DataflowRunner

def toFloat(x):
    test_list = list(map(float, x))
    return test_list

def makeRow(x):
    row = dict(zip(('v1','v2','v3','v4','v5','v6'),x))
    return row

p = beam.Pipeline(options=options) #option must be given here and when dataflowrunner is called, otherwise an error will occur. To resolve that error, some mechanism also exists.


stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv')
stage2 = stage1| "Split">>beam.Map(lambda x: x.split(','))
stage3 = stage2| "Conversion to Float">>beam.Map(toFloat)
stage4 = stage3| "Make a bigquery row">>beam.Map(makeRow)
stage5 = stage4| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='v1:FLOAT,v2:FLOAT,v3:FLOAT,v4:FLOAT,v5:FLOAT,v6:FLOAT',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    

pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [59]:
# Dataflow Runner

import apache_beam as beam
from apache_beam.runners import DataflowRunner

def toFloat(x):
    test_list = list(map(float, x))
    return test_list

def makeRow(x):
    row = dict(zip(('v1','v2','v3','v4','v5','v6'),x))
    return row

p = beam.Pipeline() #Let us remove the options here and notice the error details.

#Here is the error details
'''
raise ValueError( ValueError: Invalid GCS location: None. Writing to BigQuery with FILE_LOADS method requires a GCS location to be provided to write files to be loaded into BigQuery. 
                 Please provide a GCS bucket through custom_gcs_temp_location in the constructor of WriteToBigQuery or the fallback option --temp_location, 
                 or pass method="STREAMING_INSERTS" to WriteToBigQuery.
'''

stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv')
stage2 = stage1| "Split">>beam.Map(lambda x: x.split(','))
stage3 = stage2| "Conversion to Float">>beam.Map(toFloat)
stage4 = stage3| "Make a bigquery row">>beam.Map(makeRow)
stage5 = stage4| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table',
                                                                        custom_gcs_temp_location = 'gs://gcp-dataeng-demos-soumya/dataflow/temp', #The temp location to be input here
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='v1:FLOAT,v2:FLOAT,v3:FLOAT,v4:FLOAT,v5:FLOAT,v6:FLOAT',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    

pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip
